In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']  

start_date = '2020-01-01'
end_date = '2024-12-6'

adj_close = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

print(adj_close.tail(10))

adj_close.to_csv('adj_close_prices.csv')#save file for section use offline 



[*********************100%***********************]  5 of 5 completed

Ticker            AAPL        AMZN       GOOGL        META        MSFT
Date                                                                  
2024-11-21  228.520004  198.380005  167.630005  563.090027  412.869995
2024-11-22  229.869995  197.119995  164.759995  559.140015  417.000000
2024-11-25  232.869995  201.449997  167.649994  565.109985  418.790009
2024-11-26  235.059998  207.860001  169.119995  573.539978  427.989990
2024-11-27  234.929993  205.740005  169.229996  569.200012  422.989990
2024-11-29  237.330002  207.889999  168.949997  574.320007  423.459991
2024-12-02  239.589996  210.710007  171.490005  592.830017  430.980011
2024-12-03  242.649994  213.440002  171.339996  613.650024  431.200012
2024-12-04  243.009995  218.160004  174.369995  613.780029  437.420013
2024-12-05  243.039993  220.550003  172.639999  608.929993  442.619995


In [2]:
returns = adj_close.pct_change().dropna()

returns

Ticker,AAPL,AMZN,GOOGL,META,MSFT
Date,,,,,
2020-01-03,-0.009722,-0.012139,-0.005231,-0.005291,-0.012452
2020-01-06,0.007968,0.014886,0.026654,0.018834,0.002585
2020-01-07,-0.004703,0.002092,-0.001931,0.002164,-0.009118
2020-01-08,0.016087,-0.007809,0.007118,0.010138,0.015928
2020-01-09,0.021241,0.004799,0.010498,0.014311,0.012493
...,...,...,...,...,...
2024-11-29,0.010216,0.010450,-0.001655,0.008995,0.001111
2024-12-02,0.009523,0.013565,0.015034,0.032229,0.017759
2024-12-03,0.012772,0.012956,-0.000875,0.035120,0.000510


In [3]:
expected_returns = returns.mean()
cov_matrix = returns.cov()

print(expected_returns, cov_matrix)

Ticker
AAPL     0.001173
AMZN     0.000938
GOOGL    0.000957
META     0.001271
MSFT     0.001040
dtype: float64 Ticker      AAPL      AMZN     GOOGL      META      MSFT
Ticker                                                  
AAPL    0.000402  0.000270  0.000267  0.000322  0.000290
AMZN    0.000270  0.000516  0.000301  0.000393  0.000297
GOOGL   0.000267  0.000301  0.000417  0.000373  0.000295
META    0.000322  0.000393  0.000373  0.000807  0.000340
MSFT    0.000290  0.000297  0.000295  0.000340  0.000372


In [4]:
lambda_values = np.linspace(0.1, 1, 10)
portfolio_returns = []
portfolio_risks = []

In [5]:
# First, let's scale the returns and covariance matrix to annual values
annual_factor = 252  # Trading days in a year
expected_returns_annual = expected_returns * annual_factor
cov_matrix_annual = cov_matrix * annual_factor

# Clear previous results
portfolio_returns = []
portfolio_risks = []

for lambda_risk in lambda_values:
    # Create and solve the optimization problem
    weights = cp.Variable(n_assets)
    portfolio_return = expected_returns_annual.values @ weights
    portfolio_variance = cp.quad_form(weights, cov_matrix_annual)
    
    objective = cp.Minimize(-portfolio_return + lambda_risk * portfolio_variance)
    constraints = [
        cp.sum(weights) == 1,  # Weights must sum to 1
        weights >= 0.0001      # Minimum weight of 0.01% to avoid numerical issues
    ]
    
    problem = cp.Problem(objective, constraints)
    result = problem.solve(solver=cp.ECOS)
    
    # Add validation check
    if abs(np.sum(weights.value) - 1.0) > 1e-6:
        print(f"Warning: Weights sum to {np.sum(weights.value):.6f}")
    
    current_weights = weights.value
    current_return = float(current_weights @ expected_returns_annual.values)
    current_risk = float(np.sqrt(current_weights @ cov_matrix_annual @ current_weights))
    
    print(f"\nLambda: {lambda_risk:.2f}")
    print("Weights:", {ticker: f"{w:.4f}" for ticker, w in zip(tickers, current_weights)})
    print(f"Return: {current_return:.4%}, Risk: {current_risk:.4%}")
    
    portfolio_returns.append(current_return)
    portfolio_risks.append(current_risk)

# Modified plotting code for better visualization
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=portfolio_risks,
    y=portfolio_returns,
    mode='lines+markers',
    name='Efficient Frontier',
    hovertemplate='Risk: %{x:.2%}<br>Return: %{y:.2%}<br>λ: %{text:.2f}<extra></extra>',
    text=lambda_values
))

fig.update_layout(
    title='Portfolio Risk-Return Trade-off',
    xaxis_title='Portfolio Risk (Standard Deviation)',
    yaxis_title='Expected Portfolio Return',
    xaxis_tickformat='.2%',
    yaxis_tickformat='.2%',
    showlegend=True,
    height=600,
    width=800
)

fig.show()

NameError: name 'n_assets' is not defined

In [ ]:
optimal_weights = weights.value
opt_return = (weights.value @ expected_returns.values)
opt_risk = np.sqrt(weights.value @ cov_matrix @ weights.value)
portfolio_returns.append(opt_return)
portfolio_risks.append(opt_risk)

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=portfolio_risks,
    y=portfolio_returns,
    mode='lines+markers',
    name='Efficient Frontier',
    hovertemplate='Risk: %{x:.2%}<br>Return: %{y:.2%}<br>λ: %{text:.2f}<extra></extra>',
    text=lambda_values
))
fig.update_layout(
    title='Portfolio Risk-Return Trade-off',
    xaxis_title='Portfolio Risk (Standard Deviation)',
    yaxis_title='Expected Portfolio Return',
    xaxis_tickformat='.2%',
    yaxis_tickformat='.2%',
    showlegend=True
)
fig.show()

# add quadrtic function for non convex


In [ ]:
non_linear_constraint_exp = cp.exp(weights[2]) <= 1.5
constraints.append(non_linear_constraint_exp)

In [ ]:
problem = cp.Problem(objective, constraints)
problem.solve()


print("Optimal Weights:")
for i, ticker in enumerate(tickers):
    print(f"{ticker}: {optimal_weights[i]:.4f}")
optimal_weights_df = pd.DataFrame(optimal_weights, index=tickers, columns=['Weight'])
optimal_weights_df.to_csv('optimal_weights.csv')


Optimal Weights:
AAPL: 0.6973
MSFT: -0.0000
GOOGL: 0.0000
AMZN: 0.2059
META: 0.0968


In [ ]:
import plotly.express as px

# Load the adjusted close prices from the CSV file
adj_close = pd.read_csv('adj_close_prices.csv', index_col=0, parse_dates=True)

# Create a line plot for each stock
fig = px.line(adj_close, x=adj_close.index, y=adj_close.columns, 
              labels={'value': 'Adjusted Close Price', 'variable': 'Stock'},
              title='Stock Price Trends')

# Update layout for better visualization
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    legend_title='Stock',
    hovermode='x unified'
)

# Show the plot
fig.show()

In [ ]:
import plotly.graph_objects as go

adj_close = pd.read_csv('adj_close_prices.csv', index_col=0, parse_dates=True)

adj_close['AAPL_MA50'] = adj_close['AAPL'].rolling(window=50).mean()
adj_close['AAPL_MA200'] = adj_close['AAPL'].rolling(window=200).mean()

fig = go.Figure(data=[go.Candlestick(x=adj_close.index,
                                     open=adj_close['AAPL'].shift(1),
                                     high=adj_close['AAPL'].rolling(window=2).max(),
                                     low=adj_close['AAPL'].rolling(window=2).min(),
                                     close=adj_close['AAPL'],
                                     name='AAPL')])

fig.add_trace(go.Scatter(x=adj_close.index, y=adj_close['AAPL_MA50'], 
                         mode='lines', name='50-day MA'))
fig.add_trace(go.Scatter(x=adj_close.index, y=adj_close['AAPL_MA200'], 
                         mode='lines', name='200-day MA'))

fig.update_layout(
    title='AAPL Candlestick Chart with Moving Averages',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    legend_title='Indicator',
    hovermode='x unified'
)

fig.show()

# Return plot

In [ ]:
import plotly.express as px

returns = adj_close.pct_change().dropna()

fig_hist = px.histogram(returns, x=returns.columns, 
                        nbins=50, 
                        labels={'value': 'Daily Return', 'variable': 'Stock'},
                        title='Distribution of Daily Returns')

fig_hist.update_layout(
    xaxis_title='Daily Return',
    yaxis_title='Frequency',
    legend_title='Stock',
    hovermode='x unified'
)

fig_hist.show()

# fig_box = px.box(returns, y=returns.columns, 
#                  labels={'value': 'Daily Return', 'variable': 'Stock'},
#                  title='Box Plot of Daily Returns')

# fig_box.update_layout(
#     yaxis_title='Daily Return',
#     legend_title='Stock',
#     hovermode='x unified'
# )

# fig_box.show()

In [ ]:
import plotly.express as px
correlation_matrix = adj_close.corr()

fig_corr = px.imshow(correlation_matrix, 
                     labels=dict(color="Correlation"),
                     x=correlation_matrix.columns,
                     y=correlation_matrix.columns,
                     title='Correlation Matrix of Adjusted Close Prices')

fig_corr.update_layout(
    xaxis_title='Stock',
    yaxis_title='Stock',
    coloraxis_colorbar=dict(title="Correlation"),
    hovermode='closest'
)

fig_corr.show()

In [ ]:
import cvxpy as cp
constraints = [cp.sum(weights) == 1, weights >= 0]

# Add convex approximation for stock allocation
# Using quadratic penalty instead of exponential
quadratic_penalty = cp.quad_form(weights, np.eye(n_assets)) 
objective = cp.Minimize(-portfolio_return + lambda_risk * portfolio_variance + 0.1 * quadratic_penalty)

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Update and display optimal weights
optimal_weights = weights.value
print("\nUpdated Optimal Weights after Convex Approximation:")
for i, ticker in enumerate(tickers):
    print(f"{ticker}: {optimal_weights[i]:.4f}")



Updated Optimal Weights after Convex Approximation:
AAPL: 0.2009
MSFT: 0.1993
GOOGL: 0.1996
AMZN: 0.2000
META: 0.2002
